# Generation of ideal FCC lattice sites for close-packed surface slab model

## Header

In [ ]:
import os
import math
import numpy as np

print('This script generates a list of all possible ideal lattice sites layer-by-layer for a given unit cell of a FCC close-packed surface slab model.')
print('This script is interactive and requires user input. Please read carefully before proceeding: \n')

print('Note 1: Intended to be used as an input for LAMMPS NVT trajectory clamping (via lmpclamp.py).')
print('Note 2: Requires a LAMMPS DATA file containing equilibrated box dimensions and relaxed atomic positions.')
print('Note 3: Assumes bottommost exposed atoms as fixed.')

## Extract unit cell information

In [ ]:
pwd = os.getcwd()

# Load LAMMPS data file after NPT equilibration + ionic relaxation
name = input('Enter the name of the LAMMPS DATA file containing the equilibrated box dimensions & relaxed atomic positions : ')
file_eq = pwd + '/' + name
log = open(file_eq,'r')
log_lines = log.readlines()
log_lines = [line.split() for line in log_lines]

for line_index, line in enumerate(log_lines):
    if line:
        for l in line:
            
            if l == 'atoms':
                N_atom = int(line[0])
            
            elif l == 'xlo':
                xlo = float(line[0])
                xhi = float(line[1])
                Lx = xhi-xlo
            
            elif l == 'ylo':
                ylo = float(line[0])
                yhi = float(line[1])
                Ly = yhi-ylo
            
            elif l == 'zlo':
                zlo = float(line[0])
                zhi = float(line[1])
                Lz = zhi-zlo
            
            elif l == 'xy':
                xy = float(line[0])
                xz = float(line[1])
                yz = float(line[2])
            
            elif l == 'Atoms':
                head = line_index + 1
                start = line_index + 2
            
            elif l == 'Velocities':
                end = line_index - 1

lat = np.array([[Lx, 0.0, 0.0], [xy, Ly, 0.0], [xz, yz, Lz]])    # Lattice matrix

xyz = []    # N*3 matrix
for line in log_lines[start:end]:

    line[:2] = np.array(line[:2]).astype(int)    # Atom ID, atom type
    line[2:5] = np.array(line[2:5]).astype(float)    # x, y, z
    line[5:8] = np.array(line[5:8]).astype(int)    # ix, iy, iz (inverse sign)

    x = line[2] + line[5]*Lx + line[6]*xy    # Wrapped coordinates
    y = line[3] + line[6]*Ly
    z = line[4] + line[7]*Lz
    
    xyz.append([x,y,z])
xyz = np.array(xyz)

ID2 = int(input('Enter the ID of a nearest-neighbor atom that lies in the same layer as atom #1 : '))
ID3 = int(input('Enter the ID of another nearest-neighbor atom that lies in the same layer as atom #1 : '))
ID4 = int(input('Enter the ID of an atom that lies one layer above atom #1 : '))

for index, coord in enumerate(xyz):
    
    ID = log_lines[start+index][0]
    
    if ID == 1:
        atom1 = coord    # atom1 = Reference atom
    
    elif ID == ID2:
        atom2 = coord    # atom2 = 1st coplanar atom

    elif ID == ID3:
        atom3 = coord    # atom3 = 2nd coplanar atom
        
    elif ID == ID4:
        atom4 = coord    # atom4 = Atom one layer above

r12 = np.linalg.norm(np.subtract(atom1,atom2))
r13 = np.linalg.norm(np.subtract(atom1,atom3))
dr = np.mean([r12, r13])    # In-plane NN distance
dr_ort = 0.5*math.sqrt(3)*dr    # Orthogonal NN distance

a1 = np.subtract(atom2,atom1)    # 1st intralayer vector
a1 = a1 / np.linalg.norm(a1)

a2 = np.subtract(atom3,atom1)    # 2nd intralayer vector
a2 = a2 / np.linalg.norm(a2)

a3 = np.cross(a1,a2)    # Normal vector
a3 = a3 / np.linalg.norm(a3)

if a3[2] < 0:
    a3 = -a3    # Ensure normal vector along +z
    a3_flip = True
    a1 = np.cross(a2,a3)    # Orthogonalize intralayer vector along x
    a1 = a1 / np.linalg.norm(a1)
else:
    a3_flip = False
    a2 = np.cross(a3,a1)
    a2 = a2 / np.linalg.norm(a2)

dn = np.dot(atom4,a3) - np.dot(atom1,a3)    # Interlayer distance; n = normal component

log.close()

print('Unit cell information & normal direction extracted. \n')

## Rotate & generate reference sites

In [ ]:
if a3_flip:
    R = np.linalg.inv(np.array([a2, a1, a3]))    # Rotation matrix
else:
    R = np.linalg.inv(np.array([a1, a2, a3]))

xyz_rot = np.dot(xyz,R)    # Rotate unit cell
xyz_rot = xyz_rot[np.argsort(xyz_rot[:,2])]    # Sort by z

#file_rot = pwd + '/rot.xyz'
#rot = open(file_rot,'w')
#header_rot = '%i\n\n'%(N_atom)
#rot.write(header_rot)
#
#for atom in range(0,N_atom):
#    x = xyz_rot[atom][0]
#    y = xyz_rot[atom][1]
#    z = xyz_rot[atom][2]
#    line_rot = '%s\t%f\t%f\t%f\n'%('Ag', x, y, z)
#    rot.write(line_rot)
#
#rot.close()

N_layer = 1    # Number of layers
for atom in range(1,N_atom):
    if xyz_rot[atom][2] - xyz_rot[atom-1][2] > 0.5*dn:
        N_layer += 1

A = []    # Site A
for atom in range(0,N_atom):
    if xyz_rot[atom][2] - xyz_rot[0][2] < 0.5*dn:    # Bottommost layer fixed
        x = xyz_rot[atom][0]
        y = xyz_rot[atom][1]
        z = xyz_rot[atom][2] + dn    # Bottommost layer muted
        A.append([x,y,z])
A = np.array(A)

for atom in range(0,len(A)):    # Add +/-xy periodic images to ensure no sites are missed during replication
    
    x0 = A[atom][0]
    y0 = A[atom][1]
    z0 = A[atom][2]
    
    # 8 additional images
    A = np.append(A, [[x0+Lx, y0, z0]], axis=0)
    A = np.append(A, [[x0+Lx+xy, y0+Ly, z0]], axis=0)
    A = np.append(A, [[x0+xy, y0+Ly, z0]], axis=0)
    A = np.append(A, [[x0-Lx+xy, y0+Ly, z0]], axis=0)
    A = np.append(A, [[x0-Lx, y0, z0]], axis=0)
    A = np.append(A, [[x0-Lx-xy, y0-Ly, z0]], axis=0)
    A = np.append(A, [[x0-xy, y0-Ly, z0]], axis=0)
    A = np.append(A, [[x0+Lx-xy, y0-Ly, z0]], axis=0)

B = []    # Site B
for atom in range(0,len(A)):
    x = A[atom][0] + 0.5*dr
    y = A[atom][1] + 0.5*dr/math.sqrt(3)
    z = A[atom][2]
    B.append([x, y, z])

C = []    # Site C
for atom in range(0,len(A)):
    x = B[atom][0] + 0.5*dr
    y = B[atom][1] + 0.5*dr/math.sqrt(3)
    z = B[atom][2]
    C.append([x, y, z])

In [ ]:
file_poscar = pwd + '/ref_site.vasp'
poscar = open(file_poscar,'w')

sys = 'Pd/Ag(111)'
scale = 1.00000000000000

ax = xhi - xlo
ay = 0.0000000000000000
az = 0.0000000000000000

bx = xy
by = yhi - ylo
bz = 0.0000000000000000

cx = xz
cy = yz
cz = zhi - zlo

header_poscar = '%s\n%f\n%f %f %f\n%f %f %f\n%f %f %f\n%s\n%i\n%s\n'%(sys, scale, ax, ay, az, bx, by, bz, cx, cy, cz, 'Ag', N_atom, 'Cartesian')
poscar.write(header_poscar)

## Replicate layer-by-layer & rotate back

In [ ]:
file = pwd + '/ref_site.txt'
out = open(file,'w')

normal = '%s\t%f\t%f\t%f\n'%('Normal vector : ', a3[0], a3[1], a3[2])
out.write(normal)

header = 'x\ty\tz\tLayer #\tSite type\n'
out.write(header)

# Rotate back to original orientation
for L in range(0,N_layer+2):    # Include two extra ghost layers for the surface
    
    for atom in range(0,len(A)):
        x = A[atom][0]
        y = A[atom][1]
        z = A[atom][2] + L*dn
        coord = np.array([x, y, z])
        cart = np.dot(coord, np.linalg.inv(R))    # Cartesian
        direct = np.dot(cart, np.linalg.inv(lat))    # Direct
        
        for s_index, s in enumerate(direct):    # Prune sites lying outside unit cell
            if not (0.0 < s < 1.0):
                break
            elif s_index == len(direct)-1:
                line = '%f\t%f\t%f\t%i\t%s\n'%(cart[0], cart[1], cart[2], L+1, 'A')
                out.write(line)
                line_poscar = '%f\t%f\t%f\n'%(cart[0], cart[1], cart[2])
                poscar.write(line_poscar)
    
    for atom in range(0,len(B)):
        x = B[atom][0]
        y = B[atom][1]
        z = B[atom][2] + L*dn
        coord = np.array([x, y, z])
        cart = np.dot(coord, np.linalg.inv(R))
        direct = np.dot(cart, np.linalg.inv(lat))

        for s_index, s in enumerate(direct):
            if not (0.0 < s < 1.0):
                break
            elif s_index == len(direct)-1:
                line = '%f\t%f\t%f\t%i\t%s\n'%(cart[0], cart[1], cart[2], L+1, 'B')
                out.write(line)
                line_poscar = '%f\t%f\t%f\n'%(cart[0], cart[1], cart[2])
                poscar.write(line_poscar)
    
    for atom in range(0,len(C)):
        x = C[atom][0]
        y = C[atom][1]
        z = C[atom][2] + L*dn
        coord = np.array([x, y, z])
        cart = np.dot(coord, np.linalg.inv(R))
        direct = np.dot(cart, np.linalg.inv(lat))

        for s_index, s in enumerate(direct):
            if not (0.0 < s < 1.0):
                break
            elif s_index == len(direct)-1:
                line = '%f\t%f\t%f\t%i\t%s\n'%(cart[0], cart[1], cart[2], L+1, 'C')
                out.write(line)
                line_poscar = '%f\t%f\t%f\n'%(cart[0], cart[1], cart[2])
                poscar.write(line_poscar)

out.close()
poscar.close()

print('Reference sites generated > ref_site.txt\n')

print('lmpclamp.py can be used for LAMMPS NVT trajectory clamping.')